In [74]:
import requests
import pandas as pd
import selenium.common.exceptions
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
from selenium.webdriver.common.keys import Keys
import datetime as dt
import time
import re
import os
import numpy as np
import pymongo


# Settings
options = webdriver.ChromeOptions()
options.headless = False
options.add_argument("window-size=1920,1080")
options.add_argument('--no-sandbox')
options.add_argument('--disable-dev-shm-usage')
driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)
start_url = "https://coolparcel.com/shipping/international/shipping-calculator"
wait = WebDriverWait(driver, 1)
time1 = dt.datetime.now()
current_time = time1.strftime("%Y-%m-%dT%H_%M_%S")



====== WebDriver manager ======
Current google-chrome version is 103.0.5060
Get LATEST chromedriver version for 103.0.5060 google-chrome
Driver [C:\Users\Alibaba\.wdm\drivers\chromedriver\win32\103.0.5060.53\chromedriver.exe] found in cache
C:\Users\Alibaba\AppData\Local\Temp/ipykernel_6564/1454006962.py:26: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(ChromeDriverManager().install(), options=options)


In [75]:
def element_by_id(id1:str):
    element = driver.find_element(by=By.ID, value=id1)
    return element

In [76]:
def element_by_selector(selector:str):
    element = driver.find_element(by= By.CSS_SELECTOR, value=selector)
    return element

In [77]:
def country_info_input(country:str, postalcode:str, from1 = False):
    # Setup input field variables
    country_input = element_by_id("package-input-origin") if from1 else element_by_id("package-input-destination")
    postal_input = element_by_id("package-origin_postcode") if from1 else element_by_id("package-delivery_postcode")

    # Write country
    country_input.click()
    time.sleep(1)
    country_input.send_keys(Keys.BACKSPACE)
    time.sleep(1)
    country_input.send_keys(country)
    time.sleep(1)

    # Write postalcode
    postal_input.send_keys(postalcode)

In [78]:
def setup_search(countries:list, postalcodes:list, from1:list, package_specs:dict):
    """
    :param countries: List of origin and destination countries
    :param postalcodes: List of postalcodes from origin and destination
    :param from1: List that specifies if first or second country is origin
    :param package_specs: Example input {"weight":"20", "length":"20", "width":"30", "height":"5"} -- Weight in lb(pounds), length measurements in In(inches)
    :return
    """
    # Loop through and setup countries input fields
    for index, country in enumerate(countries):
        country_info_input(country, postalcodes[index], from1[index])
        time.sleep(1)

    # Setup measurement variables
    weight = element_by_selector("#clone-package-container > div > div > div:nth-child(1) > div > div.form-group.col-6.col-lg-6 > div > input")
    length = element_by_selector("#clone-package-container > div > div > div:nth-child(2) > div > div:nth-child(1) > div > input")
    width = element_by_selector("#clone-package-container > div > div > div:nth-child(2) > div > div:nth-child(2) > div > input")
    height = element_by_selector("#clone-package-container > div > div > div:nth-child(2) > div > div:nth-child(3) > div > input")

    # Open custom measurements
    size_button = element_by_id("package-custom-size")
    # driver.execute_script("arguments[0].scrollIntoView();", size_button)
    time.sleep(1)
    size_button.click()
    time.sleep(1)

    # Setup measurements input fields
    weight.send_keys(package_specs["weight"])
    time.sleep(1)
    length.send_keys(package_specs["length"])
    time.sleep(1)
    width.send_keys(package_specs["width"])
    time.sleep(1)
    height.send_keys(package_specs["height"])
    time.sleep(1)

    # Click search
    find_price = element_by_selector("#package > form > div.details-packages > div.find-price > button")
    find_price.click()

In [79]:
def scrape_data():
    info_container = element_by_id("resultContainer")
    data = info_container.text
    print(data)

In [80]:
def main():
    time.sleep(3)
    driver.get(start_url)
    time.sleep(10)
    setup_search(["Denmark", "Finland"], ["8000", "00100"], [True, False], {"weight":"20", "length":"20", "width":"30", "height":"5"})
    time.sleep(20)
    scrape_data()

In [81]:
main()

Door → Door
DHL Express Worldwide
1 Business Day
$265
TAX FREE
Book Now
PICKUP
AVAILABLE
ANY ADDRESS
DELIVERY
BUSINESS HOURS
BEFORE 11:59 AM
PRINTER
REQUIRED
Door → Door
International Economy
4 Business Days
$265
TAX FREE
Book Now
PICKUP
AVAILABLE
ANY ADDRESS
DELIVERY
BUSINESS HOURS
BEFORE 6:00 PM
PRINTER
REQUIRED
Door → Door
UPS Standard
3 Business Days
$305
TAX FREE
Book Now
PICKUP
AVAILABLE
ANY ADDRESS
DELIVERY
BUSINESS HOURS
BEFORE 5:00 PM
PRINTER
REQUIRED
Door → Door
Fedex International Priority
1 Business Day
$380
TAX FREE
Book Now
PICKUP
AVAILABLE
ANY ADDRESS
DELIVERY
BUSINESS HOURS
BEFORE 6:00 PM
PRINTER
REQUIRED
Door → Door
Fedex International Priority Express
1 Business Day
$405
TAX FREE
Book Now
PICKUP
AVAILABLE
ANY ADDRESS
DELIVERY
BUSINESS HOURS
BEFORE 12:00 PM
PRINTER
REQUIRED
Door → Door
DHL Economy Select
5 Business Days
$594
TAX FREE
Book Now
PICKUP
AVAILABLE
ANY ADDRESS
DELIVERY
BUSINESS HOURS
BEFORE 11:59 AM
PRINTER
REQUIRED
Door → Door
UPS Worldwide Saver
1 Business